# Classificando nomes com uma *Character-Level RNN*

https://github.com/spro/practical-pytorch/blob/master/char-rnn-classification/char-rnn-classification.ipynb

---



### Problema: Dado um nome próprio de entrada, classificar esse nome de acordo com a nacionalidade a que ele pertence.

python predict.py **Hinton**

(-0.47) Scottish

(-1.52) English

(-3.57) Irish


-

python predict.py **Schmidhuber**

(-0.19) German

(-2.48) Czech

(-2.68) Dutch


---
### Import de bibliotecas

In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
  
import unicodedata
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
import sys, random, os
import matplotlib.pyplot as plt

%matplotlib inline


---

### Importando dataset

In [0]:
# comente as duas linhas seguintes caso rode mais de uma vez
!wget https://download.pytorch.org/tutorial/data.zip #
!unzip data.zip #
############################################################

root_path = 'data/names/'
all_filenames = []
for file_name in os.listdir(root_path):
  all_filenames.append(os.path.join(root_path,file_name))

In [0]:
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename).read().strip().split('\n')
    #return lines
    return [unicodedata.normalize('NFKD', line).encode('ascii', 'ignore') for line in lines]

for filename in all_filenames:
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)
print('n_categories =', n_categories)

In [0]:
category_lines['Italian'][:5]


---
### Convertendo caracteres para tensores
Nesse caso, cada caracter será convertido para uma representação *one-hot*


In [0]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letter_to_tensor(letter):
    tensor = torch.zeros(1, n_letters)
    letter_index = all_letters.find(letter)
    tensor[0][letter_index] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def line_to_tensor(line):
    tensor = torch.zeros(len(line), n_letters)
    for li, letter in enumerate(line.decode('utf-8')):
        letter_index = all_letters.find(letter)
        tensor[li][letter_index] = 1
    return tensor

In [0]:
print(letter_to_tensor('a'))



---


### Funções auxiliares para estruturar as amostras de treino e teste

In [0]:
def category_from_output(output):
    top_n, top_i = output.data.topk(1) # Tensor out of Variable with .data
    category_i = top_i[0][0]
    return all_categories[category_i]

In [0]:
from sklearn.model_selection import train_test_split as split

X_train, X_test = {}, {}
for category in all_categories:
  train, test = split(category_lines[category], test_size=0.1)
  
  X_train[category] = train
  X_test[category] = test



---

### Construindo modelo RNN

* Implemente um modelo para classificação de nomes próprios (série de caracteres) usando apenas camadas *RNNCell*, *Linear* e ativação *LogSoftmax*
* Cada entrada (caracter) possui dimensão (57): alfabeto maiúsculo e minúsculo
* *Hidden size* possui dimensão (256): hiperparâmetro 
* Saída possui dimensão (18): vetor de probabilidade de classes
* Batch size = 1 de acordo com a implementação do loop de treinamento

**Links úteis**

RNNCell: https://pytorch.org/docs/stable/nn.html#torch.nn.RNNCell

Linear: https://pytorch.org/docs/stable/nn.html#torch.nn.Linear

Non-linear activations: https://pytorch.org/docs/stable/nn.html#non-linear-activations-other



In [0]:
class CustomNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, nonlinearity='tanh'):
        super(CustomNetwork, self).__init__()
        

    
    def forward(self, input):
      
      # Set initial hidden state 
      #self.hidden = Variable( ).double()
      self.hidden = Variable( ''' Adicione aqui a inicialização do hidden state ''' ).double().cuda() #GPU
        

n_hidden = 256
model = CustomNetwork(n_letters, n_hidden, n_categories)
model.double().cuda() #GPU



---

### Instanciando *Loss* e Otimizador

In [0]:
criterion = nn.NLLLoss().cuda() #GPU
learning_rate  = 0.0002 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



---

### Treinando / Testando modelo

In [0]:
num_epochs = 20
num_steps  = 2000
print_every = 2000

current_loss = 0
current_acc = 0
all_losses, all_val_losses, all_acc, all_val_acc = [], [], [], []
for epoch in range(num_epochs):  

  for step in range(num_steps):
    
    # Set to Train Mode
    model.train()
    
    category = random.choice(all_categories)
    line = random.choice(X_train[category])
  
    #category_tensor = Variable(torch.LongTensor([all_categories.index(category)]))
    #line_tensor     = Variable(line_to_tensor(line))
    category_tensor = Variable(torch.LongTensor([all_categories.index(category)])).cuda() #GPU
    line_tensor     = Variable(line_to_tensor(line)).double().cuda() #GPU

    # Forward pass
    output = model(line_tensor)
    loss = criterion(output, category_tensor)
    current_loss += loss
    current_acc  += 1 if category_from_output(output) == category else 0
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (step+1) % print_every == 0:
      
      #Set to evaluation mode
      model.eval()
      n_test_steps = 100
      
      test_loss, acc = 0, 0
      for tstep in range(n_test_steps):
      
        category = random.choice(all_categories)
        line = random.choice(X_test[category])

        #category_tensor = Variable(torch.LongTensor([all_categories.index(category)]))
        #line_tensor     = Variable(line_to_tensor(line))
        category_tensor = Variable(torch.LongTensor([all_categories.index(category)])).cuda() #GPU
        line_tensor     = Variable(line_to_tensor(line)).double().cuda() #GPU

        output = model(line_tensor)
        loss = criterion(output, category_tensor)
        test_loss += loss
        acc += 1 if category_from_output(output) == category else 0
      
      
      current_loss = current_loss.data[0]/float(print_every)
      current_acc  = current_acc/float(print_every)
      test_loss    = test_loss.data[0]/float(n_test_steps)
      acc          = acc/float(n_test_steps)
      
      print('\rEpoch: {0} Train Loss: {2} Train Acc: {3}% Val Loss: {4}  Val Acc: {5}%'.format(epoch+1, step, current_loss, current_acc*100, test_loss, acc*100))

      all_losses.append(current_loss)
      all_acc.append(current_acc)
      all_val_losses.append(test_loss)
      all_val_acc.append(acc)

      current_loss = 0
      current_acc  = 0
  



---


### Análise e plots

In [0]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,3))


ax1.plot(all_losses[1:], label='Train')
ax1.plot(all_val_losses[1:], label='Test')
ax1.set_title('Model Convergence - Loss')
ax1.set_xlabel('epochs')
ax1.set_ylabel('Loss')
ax1.legend()

ax2.plot(all_acc, label='Train')
ax2.plot(all_val_acc, label='Test')
ax2.set_title('Model Convergence - Accuracy')
ax2.set_xlabel('epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()



---

### Fazendo predições com entradas de usuário

In [0]:
def predict_input(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    
    tensor = torch.zeros(len(input_line), n_letters)
    for li, letter in enumerate(input_line):
        letter_index = all_letters.find(letter)
        tensor[li][letter_index] = 1
        
    output = model(Variable(tensor).double().cuda())

    # Get top N categories
    topv, topi = output.data.topk(n_predictions, 1, True)
    predictions = []

    for i in range(n_predictions):
        value = topv[0][i]
        category_index = topi[0][i]
        print('(%.2f) %s' % (value, all_categories[category_index]))
        predictions.append([value, all_categories[category_index]])

predict_input('Mbappe')
predict_input('Guillermo')
predict_input('Kyle')